## This notebook is specifically plotting the error maps 

##### Tile and year selection

Choose which tile (see MODIS grid) and which year. Reference the grid image. 

The `h` followed by two numerical digits represent the <b>horizontal</b> tile ID. Use the column space to determine this ID. 

The `v` followed by two numerical digits represent the <b>vertical</b> tile ID. Use the row space to determine this ID. 

For example, the tile that is 9 columns to the right and 5 rows down is `h09v05`.

Example:
```python
TILE = 'h09v05'
```

![MODIS Grid Overlay](../imgs/modis_overlay.png)

In [1]:
# !pip install localtileserver

In [2]:
from ipyleaflet import Map, Marker, basemaps, ScaleControl, LayersControl
from localtileserver import TileClient, get_leaflet_tile_layer 
from ipyleaflet import LegendControl, FullScreenControl, Popup
import matplotlib.colors as mcolors
from ipysheet import from_dataframe
import matplotlib.pyplot as plt
import ipywidgets as widgets
import rioxarray as rxr
from osgeo import gdal
from glob import glob
import seaborn as sns
import pyarrow as pa
import xarray as xr
import numpy as np
import tempfile
import ipysheet
import warnings
import joblib
import os


os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = \
    f"{os.environ['JUPYTERHUB_SERVICE_PREFIX'].lstrip('/')}/proxy/{{port}}"

### Data Parameters

In [3]:
TILE = 'h12v09'
# TILE = 'h22v01'
# TILE = 'h21v10'
# TILE = 'h09v05'

In [4]:
YEAR = 2019

In [5]:
target = False #True
match = False
noc = False #True
eb = True #True

In [11]:
############
#Directory Paths
############
#RFA dir paths

data_dir = '/explore/nobackup/projects/ilab/data/MODIS/PRODUCTION/Burke_MW_RFA'

outlier_pts_dir = f'{data_dir}/no_outlier_pts'
outlier_cluster_dir = f'{data_dir}/no_outlier_cluster'

clus_TAR_PATH = f'{outlier_cluster_dir}/v4_sample' #targeted rfa trained using v4.2.1 data
clus_EB_PATH = f'{outlier_cluster_dir}/v2_cluster' #even balance trained rfa
clus_MATCH_PATH = f'{outlier_cluster_dir}/v2_match' #random subset match size of even balance rfa
clus_NOC_PATH = f'{outlier_cluster_dir}/v2_total' #rfa trained using v2.0.1 data

pts_TAR_PATH = f'{outlier_pts_dir}/v4_sample' #targeted rfa trained using v4.2.1 data
pts_EB_PATH = f'{outlier_cluster_dir}/v2_cluster_all_land' #even balance trained rfa
pts_MATCH_PATH = f'{outlier_pts_dir}/v2_match' #random subset match size of even balance rfa
pts_NOC_PATH = f'{outlier_pts_dir}/v2_total' #rfa trained using v2.0.1 data


#C6 dir paths
MOD44W_C6_BASEPATH = '/explore/nobackup/people/mcarrol2/MODIS_water/v5_outputs/'
CACHE_DIR = '.cache'
os.makedirs(CACHE_DIR, exist_ok=True)

############
#Specific File Paths
############
#RFA file paths
file_qa_path = f'*{YEAR}*{TILE}*ProductQA.*tif'
file_path = f'*{YEAR}*{TILE}*Product.*tif'
file_sumobs_path = f'*{YEAR}*{TILE}*SumWater*tif'
file_ogmask_path = f'*{YEAR}*{TILE}*-Mask*tif'
#C6 file paths
mod44w_c6_path = f'{MOD44W_C6_BASEPATH}/{str(YEAR)}/MOD44W_{TILE}_{YEAR}_v5.tif'
print(mod44w_c6_path)
if not os.path.exists(mod44w_c6_path):
    raise FileNotFoundError(f'Could not find the MOD44W C6 file: {mod44w_c6_path}')

############
#CMAPS and legend
############
water_c6_cmap: list = [mcolors.cnames['white'], mcolors.cnames['white']]
water_rfa_qa_cmap: list = ['#ee82ee', '#FCB900', '#FF6900', '#800080']
qa_water_legend_dict = {
    'QA- Perm Water Flipped L->W': '#ee82ee', 
    'QA- Ocean Mask L->W': '#FCB900',
    'QA- Burn Scar W->L': '#FF6900',
    'QA- DEM Slope Change W->L': '#800080'}


############
#Geotransform parameters
############
crs = 'PROJCS["Sinusoidal",GEOGCS["Sphere",DATUM["Sphere",SPHEROID["Sphere",6371000,0]],PRIMEM["Greenwich",0],' + \
    'UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Sinusoidal"]' + \
    ',PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0]' + \
    ',UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'
mod44w_cs_ds = gdal.Open(mod44w_c6_path)
transform = mod44w_cs_ds.GetGeoTransform()

############
#KWARGS
############
temporary_files = []
rfa_kwargs = {'nodata':0,'show':False,'vmin':0,'vmax':1,'max_zoom':20}
diff_kwargs = {'nodata':0,'show':False,'vmin':-1,'vmax':1,'max_zoom':20}
obs_kwargs = {'nodata':0,'show':False,'vmin':0,'vmax':365,'max_zoom':20}  
pw_kwargs = {'nodata':0,'show':False,'vmin':1,'vmax':4,'max_zoom':20}  
transform_kwargs = {'transform':transform, 'projection':crs, 'year':YEAR, 
                        'tile':TILE, 'files_to_rm':temporary_files}

/explore/nobackup/people/mcarrol2/MODIS_water/v5_outputs//2019/MOD44W_h12v09_2019_v5.tif


### Plots

#### Functions

In [7]:
def parse_qa(qa_array):
    qa_array_parsed = xr.where(qa_array == 0, 0, -1)
    qa_array_parsed = xr.where(qa_array == 4, 1, qa_array_parsed)
    qa_array_parsed = xr.where(qa_array == 6, 2, qa_array_parsed)
    qa_array_parsed = xr.where(qa_array == 9, 3, qa_array_parsed)
    return qa_array_parsed

def parse_fix_qa(qa_array):
    qa_array_parsed = xr.where(qa_array == 2, 1, 0)
    qa_array_parsed = xr.where(qa_array == 4, 2, qa_array_parsed)
    qa_array_parsed = xr.where(qa_array == 6, 3, qa_array_parsed)
    qa_array_parsed = xr.where(qa_array == 9, 4, qa_array_parsed)
    return qa_array_parsed

def open_and_write_temp(data_array, transform, projection, 
    year, tile, name = None, files_to_rm = None) -> str:
    tmpdir = tempfile.gettempdir()
    name_to_use = data_array.name if not name else name
    tempfile_name = f'MOD44W.A{year}001.{tile}.061.{name_to_use}.tif'
    tempfile_fp = os.path.join(tmpdir, tempfile_name)
    # print(glob(tempfile_fp))
    if os.path.exists(tempfile_fp): 
        os.remove(tempfile_fp)
    tempfile_fp = os.path.join(tmpdir, tempfile_name)
    print(tempfile_fp)
    driver = gdal.GetDriverByName('GTiff')
    outDs = driver.Create(tempfile_fp, 4800, 4800, 
                          1, gdal.GDT_Float32, 
                          options=['COMPRESS=LZW'])
    outDs.SetGeoTransform(transform)
    outDs.SetProjection(projection)
    outBand = outDs.GetRasterBand(1)
    try:
        outBand.WriteArray(data_array.data[0, :, :])
    except:
        outBand.WriteArray(data_array[0, :, :])
    outBand.SetNoDataValue(250)
    outDs.FlushCache()
    outDs = None
    outBand = None
    driver = None
    return tempfile_fp

def get_location(cache_dir: str, tile: str, def_location: list) -> list:
    cache_fp = os.path.join(cache_dir, f'{tile}.marker.location.sv')
    if os.path.exists(cache_fp):
        location = joblib.load(cache_fp)
    else:
        location = def_location
    return location

def cache_location(tile: str, location: list) -> None:
    cache_fp = os.path.join(CACHE_DIR, f'{tile}.marker.location.sv')
    output = joblib.dump(location, cache_fp)
    return None

def initialize_marker(tile: str, location: list, cache_dir: str) -> Marker:
    name = 'Location Marker'
    title = name
    location = get_location(cache_dir, tile, location)
    marker = Marker(name=name, title=name, location=location)
    return marker

def initialize_message(location: list) -> widgets.HTML:
    ll_message = widgets.HTML()
    ll_message.value = str(location)
    return ll_message

In [8]:
def data_plot(product_path, legend_name, error_path, data_color):
    """
    In 
        product_path (str): path to the tif rf prediction files
        legend_name (str): name of the product that is plotted
        error (str) : path to the eror file
        data_color (str): the cmap color that the data will be plotted in
        
    
    Returns
        rfa_water_mask_layer (leaflet_tile_layer): Map layer of the RFA Product 
        rfa_legend_dict (dict): Dictionary of the RFA Product cmap
        
    """
    
    data_array = rxr.open_rasterio(sorted(glob(product_path))[0])
    data_qa_file = open_and_write_temp(data_array, name=f'{legend_name}_mask', **transform_kwargs)
    data_qa_array = rxr.open_rasterio(data_qa_file)
    
    if error_path is None: 
        tile_name = f'{legend_name}'
        tile_cmap = [mcolors.cnames[data_color], mcolors.cnames[data_color]]
        data_client = TileClient(data_qa_file)
        data_layer = get_leaflet_tile_layer(data_client,                         
            cmap=tile_cmap, name=tile_name, **rfa_kwargs)
        return data_layer
    
    mod44w_array = rxr.open_rasterio(sorted(glob(error_path))[0])
    
    diff_array = data_qa_array.data-mod44w_array.data
    diff_qa_file = open_and_write_temp(diff_array, name=f'{legend_name}_diff_mask', **transform_kwargs)    
    diff_client = TileClient(diff_qa_file)
    
    cm = [mcolors.cnames['white'],mcolors.cnames['white'],mcolors.cnames[data_color]]
    
    diff_layer = get_leaflet_tile_layer(
            diff_client, cmap=cm,
            name=f'{legend_name}', **diff_kwargs)
    return diff_layer

#### Using Functions

In [9]:
#c6 data
c6_client = TileClient(mod44w_c6_path)
c6_water_mask_layer = get_leaflet_tile_layer( c6_client, cmap=water_c6_cmap, name=f'C6', **rfa_kwargs)
c6_legend_dict = {'C6': mcolors.cnames['blue']}

In [12]:

if target: 
    tar_diff_layer = data_plot(f'{out_TAR_PATH}/{file_path}','v421',mod44w_c6_path,'blue') #'goldenrod')
    nooutp_tar = data_plot(f'{no_out_TAR_PATH}/{file_path}','other v421',mod44w_c6_path,'pink') #'
if eb: 
    # eb_diff_layer = data_plot(f'{clus_EB_PATH}/{file_path}','Cluster Diff', mod44w_c6_path,'blue') #'plum')
    eb_layer = data_plot(f'{clus_EB_PATH}/{file_path}','Cluster', None,'plum')
    eb_pts = data_plot(f'{pts_EB_PATH}/{file_path}','EB Pts',None,'yellow') #'
if match:
    # mat_diff_layer = data_plot(f'{clus_MATCH_PATH}/{file_path}','Match',mod44w_c6_path,'blue') #'darkorchid')
    mat_diff_layer = data_plot(f'{clus_MATCH_PATH}/{file_path}','Match',None,'blue') #'darkorchid')
    mat_pts = data_plot(f'{pts_MATCH_PATH}/{file_path}','Match Pts',None,'orange') #'darkorchid')
if noc:  
    noc_diff_layer = data_plot(f'{out_NOC_PATH}/{file_path}','v201',mod44w_c6_path,'blue') #'black')
    nooutp_noc = data_plot(f'{no_out_NOC_PATH}/{file_path}','other v201',mod44w_c6_path,'pink') #'
    

/tmp/MOD44W.A2019001.h12v09.061.Cluster_mask.tif
/tmp/MOD44W.A2019001.h12v09.061.EB Pts_mask.tif


In [13]:
#########
#Setup map
#########
m = Map(
    center=c6_client.center(),
    zoom=c6_client.default_zoom,
    basemap=basemaps.Esri.WorldImagery,
    scroll_wheel_zoom=True,
    keyboard=True,
    layout=widgets.Layout(height='600px')
)
marker_location = c6_client.center()
marker = initialize_marker(tile=TILE, location=marker_location, cache_dir=CACHE_DIR)
latlon_message = initialize_message(marker.location)

def handle_click(**kwargs):
    latlon_message.value = str(marker.location)
    marker.popup = latlon_message
    cache_location(tile=TILE, location=marker.location)

# m.add_layer(marker)

if target:
    m.add_layer(nooutp_tar)
    m.add_layer(tar_diff_layer)
    
if match: 
    m.add_layer(mat_pts)
    m.add_layer(mat_diff_layer)

if noc:
    m.add_layer(nooutp_noc)
    m.add_layer(noc_diff_layer)
    
if eb:
    # m.add_layer(eb_diff_layer)
    m.add_layer(eb_pts)
    m.add_layer(eb_layer)
    
# m.add_layer(c6_water_mask_layer)
    
marker.on_click(handle_click)
# m.add_control(legend)
m.add_control(ScaleControl(position='bottomleft'))
m.add_control(LayersControl(position='topright'))
m.add_control(FullScreenControl())

In [14]:
display(m)

Map(center=[-5.000005456878516, -55.46285991047675], controls=(ZoomControl(options=['position', 'zoom_in_text'…